# Kata10: Manejo de excepciones
### Uso de tracebacks para buscar errores
Las excepciones en Python son una característica principal del lenguaje. Es posible que te sorprenda leer que algo que genera errores se resalta como una característica. Esta sorpresa puede deberse a que las herramientas de software sólidas no parecen bloquearse con un traceback seguimiento (varias líneas de texto que indican cómo se inició y finalizó el error).

Sin embargo, las excepciones son útiles porque ayudan en la toma de decisiones generando mensajes de error descriptivos. Pueden ayudarte a controlar los problemas esperados e inesperados.

### Tracebacks
Un traceback es el cuerpo del texto que puede apuntar al origen (y al final) de un error no controlado. Comprender los componentes de un traceback hará que seas más eficaz al corregir errores o depurar un programa que no funciona bien.

La primera vez que encuentres excepciones en Python podrías verte tentado/a a evitar el error suprimiéndolo. Cuando un programa sufre un error no controlado, aparece un traceback como salida. Como verás en este módulo, los tracebacks son útiles. Hay maneras de controlar correctamente los errores para que no aparezcan o muestren información útil.

Si intentamos en un notebook, abrir un archivo inexistente sucede lo siguiente:

>>> open("/path/to/mars.jpg")
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
FileNotFoundError: [Errno 2] No such file or directory: '/path/to/mars.jpg'

Esa salida tiene varias partes clave. En primer lugar, el traceback menciona el orden de la salida. Después, informa de que el archivo es stdin (entrada en el terminal interactivo) en la primera línea de la entrada. El error es FileNotFoundError (el nombre de excepción), lo que significa que el archivo no existe o quizás el directorio correspondiente no existe.

Es mucha información. Puede ser difícil entender por qué la línea 1 es significativa o qué significa Errno 2.

Intenta crear un archivo de Python y asígnale el nombre open.py, con el contenido siguiente:

In [16]:
def main():
    open("/path/to/mars.jpg")

if __name__ == '__main__':
    main()

FileNotFoundError: [Errno 2] No such file or directory: '/path/to/mars.jpg'

### Controlando las excepciones
### Try y Except de los bloques
Vamos a usar el ejemplo de navegador a fin de crear código que abra archivos de configuración para la misión de Marte. Los archivos de configuración pueden tener todo tipo de problemas, por lo que es fundamental notificarlos con precisión cuando se presenten. Sabemos que, si no existe un archivo o directorio, se genera FileNotFoundError. Si queremos controlar esa excepción, podemos hacerlo con un bloque try y except:

In [17]:
def main():
    try:
        open("config.txt")
    except FileNotFoundError:
        print('No se puede encontrar el archivo config.txt')

if __name__ == '__main__':
    main()

No se puede encontrar el archivo config.txt


Aunque es común un archivo que no existe, no es el único error que podemos encontrar. Los permisos de archivo no válidos pueden impedir la lectura de un archivo, incluso si este existe. Vamos a crear un archivo de Python denominado config.py. El archivo tiene código que busca y lee el archivo de configuración del sistema de navegación:

A continuación, quita,ps el archivo config.txt y creamos un directorio denominado config.txt. Intentaremos llamar al archivo config.py para ver un error nuevo que debería ser similar al siguiente:

In [20]:
def main():
    try:
        open("config.txt")
    except FileNotFoundError:
        print('No se puede encontrar el archivo config.txt')

if __name__ == '__main__':
    main()

IsADirectoryError: [Errno 21] Is a directory: 'config.txt'

Una manera poco útil de controlar este error sería detectar todas las excepciones posibles para evitar un traceback. Para comprender por qué detectar todas las excepciones es problemático, probaremos actualizando la función main():

In [21]:
def main():
    try:
        open("config.txt")
    except Exception:
        print('No se puede encontrar el archivo config.txt')

if __name__ == '__main__':
    main()

No se puede encontrar el archivo config.txt


El problema ahora es que el mensaje de error es incorrecto. El archivo existe, pero tiene permisos diferentes y Python no puede leerlo. Cuando se trata con errores de software, puede resultar frustrante tener errores que hagan lo siguiente:

No indiquen cuál es el problema real.
Proporcionen una salida que no coincida con el problema real.
No sugieran lo que se puede hacer para corregir el problema.
Vamos a corregir este fragmento de código para abordar todas estas frustraciones. Revertiremos la detección de FileNotFoundError y luego agregamos otro bloque except para detectar PermissionError:

In [22]:
def main():
    try:
        open("config.txt")
    except FileNotFoundError:
        print('No se puede encontrar el archivo config.txt')
    except IsADirectoryError:
        print('config.txt encontrado pero es un directorio, no se puede leer.')

if __name__ == '__main__':
    main()

config.txt encontrado pero es un directorio, no se puede leer.


Cuando los errores son de una naturaleza similar y no es necesario controlarlos individualmente, puedes agrupar las excepciones como una usando paréntesis en la línea except. Por ejemplo, si el sistema de navegación está bajo cargas pesadas y el sistema de archivos está demasiado ocupado, tiene sentido detectar BlockingIOError y TimeOutError juntos:

In [23]:
def main():
    try:
        open("config.txt")
    except FileNotFoundError:
        print('No se puede encontrar el archivo config.txt')
    except IsADirectoryError:
        print('config.txt encontrado pero es un directorio, no se puede leer.')
    except (BlockingIOError, TimeoutError):
        print("Sobrecarga de archivos, no se puede completar la lectura del archivo.")

if __name__ == '__main__':
    main()

config.txt encontrado pero es un directorio, no se puede leer.


##### Sugerencia

Aunque puedes agrupar excepciones, solo debes hacerlo cuando no sea necesario controlarlas individualmente. Evita agrupar muchas excepciones para proporcionar un mensaje de error generalizado.

Si necesitas acceder al error asociado a la excepción, debes actualizar la línea except para incluir la palabra clave as. Esta técnica es práctica si una excepción es demasiado genérica y el mensaje de error puede ser útil:

In [24]:
def main():
    try:
        open("configuracion.txt")
    except FileNotFoundError as err:
        print('Problema al tratar de leer el archivo: ', err)

if __name__ == '__main__':
    main()

Problema al tratar de leer el archivo:  [Errno 2] No such file or directory: 'configuracion.txt'


En este caso, as err significa que err se convierte en una variable con el objeto de excepción como valor. Después, usa este valor para imprimir el mensaje de error asociado a la excepción. Otra razón para usar esta técnica es acceder directamente a los atributos del error. Por ejemplo, si detecta una excepción OSError más genérica, que es la excepción primaria de FilenotFoundError y PermissionError, podemos diferenciarlas mediante el atributo .errno:

In [25]:
def main():
    try:
        open("configuracion.txt")
    except OSError as err:
        if err.errno == 2:
            print('No se puede encontrar el archivo configuracion.txt')
        elif err.errno == 13:
            print('config.txt encontrado pero es un directorio, no se puede leer.')

if __name__ == '__main__':
    main()

No se puede encontrar el archivo configuracion.txt


### Generación de excepciones

Ahora que tienes una buena comprensión de los tracebacks y el control de excepciones, vamos a revisar la generación de excepciones.

Es posible que ya conozcas una situación que podría provocar una condición de error al escribir código. En estas situaciones, resulta útil generar excepciones que permitan que otro código comprenda cuál es el problema.

La generación de excepciones también puede ayudar en la toma de decisiones para otro código. Como hemos visto antes, en función del error, el código puede tomar decisiones inteligentes para resolver, solucionar o ignorar un problema.

Los astronautas limitan su uso de agua a unos 11 litros al día. Vamos a crear una función que, con base al número de astronautas, pueda calcular la cantidad de agua quedará después de un día o más:

In [30]:
def reserva_agua(astronautas, reserva_agua, dias):
    litros_por_dia = astronautas * 11
    total_agua_requerida = dias * litros_por_dia
    total_agua_reserva = reserva_agua - total_agua_requerida
    return f"Total agua en reserva por {dias} días es: {total_agua_reserva} litros"

PruProbemos con cinco astronautas, 100 litros de agua sobrante y dos días:

In [31]:
print(reserva_agua(5,100,2))

Total agua en reserva por 2 días es: -10 litros


Esto no es muy útil, ya que una carencia en los litros sería un error. Después, el sistema de navegación podría alertar a los astronautas que no habrá suficiente agua para todos en dos días. Si eres un ingeniero(a) que programa el sistema de navegación, podrías generar una excepción en la función water_left() para alertar de la condición de error:

In [34]:
def reserva_agua2(astronautas, reserva_agua, dias):
    litros_por_dia = astronautas * 11
    total_agua_requerida = dias * litros_por_dia
    total_agua_reserva = reserva_agua - total_agua_requerida
    if total_agua_reserva < 0:
        raise RuntimeError(f"No alcanza el agua para {astronautas} astronautas para {dias} días.")
    return f"Total agua en reserva por {dias} días es: {total_agua_reserva} litros"

Ahora volvemos a ejecutarlo

In [35]:
print(reserva_agua2(5,100,2))

RuntimeError: No alcanza el agua para 5 astronautas para 2 días.

En el sistema de navegación, el código para señalar la alerta ahora puede usar RuntimeError para generar la alerta:

In [37]:
try:
    print(reserva_agua2(5,100,2))
except RuntimeError as err:
    print(err)

No alcanza el agua para 5 astronautas para 2 días.


La función water_left() también se puede actualizar para evitar el paso de tipos no admitidos. Intentenis pasar argumentos que no sean enteros para comprobar la salida de error:

In [38]:
print(reserva_agua2(5,100,None))

TypeError: unsupported operand type(s) for *: 'NoneType' and 'int'

El error de TypeError no es muy descriptivo en el contexto de lo que espera la función. Actualizaremos la función para que use TypeError, pero con un mensaje mejor:

In [39]:
def reserva_agua3(astronautas, reserva_agua, dias):
    for argumento in [astronautas, reserva_agua, dias]:
        try:
            #Si el argumento es un número, puede realizar la siguiente operación
            argumento / 10
        except:
            #TypeError se genera solo cuando no es el tipo correcto
            #Genera la misma excepción pero con un mensaje mejor
            raise RuntimeError(f"Todos los argumentos deben ser númericos, pero recibe: {argumento}")
    litros_por_dia = astronautas * 11
    total_agua_requerida = dias * litros_por_dia
    total_agua_reserva = reserva_agua - total_agua_requerida
    if total_agua_reserva < 0:
        raise RuntimeError(f"No alcanza el agua para {astronautas} astronautas para {dias} días.")
    return f"Total agua en reserva por {dias} días es: {total_agua_reserva} litros"

Ahora volvemos a intentarlo para obtener un error mejor:

In [41]:
print(reserva_agua3(5,100,None))

RuntimeError: Todos los argumentos deben ser númericos, pero recibe: None

In [42]:
try:
    print(reserva_agua3(5,100,None))
except RuntimeError as err:
    print(err)

Todos los argumentos deben ser númericos, pero recibe: None


### Resumen
Para ser un desarrollador eficaz, debes saber cómo funcionan las excepciones y cómo controlarlas. En este módulo, has descubierto cómo usar la salida de excepción para la depuración, cómo detectar y generar excepciones y, por último, cómo afecta a la lógica de un programa cuando se producen excepciones.

Con los conocimientos del uso y el control de excepciones, puedes evitar problemas comunes, como detectarlos todos para evitar tracebacks.

#### En este módulo, has obtenido información sobre estas aptitudes:

    Leer y usar la salida de error de las excepciones
    Controlar correctamente las excepciones
    Generar excepciones con mensajes de error útiles
    Usar excepciones para controlar el flujo de un programa

Curso Propedútico de Python para Launch X - Innovacción Virtual.